<a href="https://colab.research.google.com/github/Nageswar-Sahoo/PyTorch_101/blob/main/Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
# Pytorch's tensors are similar to Numpy's ndarrays  Python Upodated Nageshwar 
!pip install torch

In [16]:
import torch.nn.functional as F
import torch.optim as optim
import torch 
import torch
import torchvision # provide access to datasets, models, transforms, utils, etc
import torchvision.transforms as transforms
from torch.utils.data import Dataset
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import random



In [18]:
train_set = torchvision.datasets.FashionMNIST(
    root='./data',
    train=True,
    download=True,
    transform = transforms.Compose([
          transforms.ToTensor()
    ])
)

In [19]:
def getRandomNumber(batchsize):
  return random.sample(range(0, 10), batchsize)

In [20]:
def get_num_correct(preds, labels):
  return preds.argmax(dim=1).eq(labels).sum().item()

In [21]:

class Network(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5) 
    self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)
    self.fc1 = nn.Linear(in_features=232, out_features=120)
    self.fc2 = nn.Linear(in_features=120, out_features=60)
    self.out = nn.Linear(in_features=60, out_features=29)

    self.fcr1 = nn.Linear(in_features=10, out_features=20)
    self.fcr2 = nn.Linear(in_features=20, out_features=40)
  
  def forward(self, t, randomNum):
    # input layer
    x = t
    y=randomNum


    # conv1 layer
    x = self.conv1(x)
    x = F.relu(x)
    x = F.max_pool2d(x, kernel_size=2, stride=2) # 28 | 24 | 12

    # conv2 layer
    x = self.conv2(x)
    x = F.relu(x)
    x = F.max_pool2d(x, kernel_size=2, stride=2) # 12 | 8 | 4 >> 12x4x4

    # reshapre
    x = x.reshape(-1, 12 * 4 * 4)
    y = y.reshape(-1, 10)

    #print(x.shape)
    #print(y.shape)

    # fcr layer
    y = self.fcr1(y)
    y = F.relu(y)


    # fcr layer
    y = self.fcr2(y)
    y = F.relu(y)


    #print("Before concat")
    #print(x.shape)
    #print(y.shape)

    x=torch.cat((x,y ), 1)

    #print("Shape After concat ")
    #print(x.shape)


    # fc1 layer
    x = self.fc1(x)
    x = F.relu(x)

    # fc2 layer
    x = self.fc2(x)
    x = F.relu(x)




    # output layer
    x = self.out(x)
    # x = F.softmax(x, dim=1)
    return x

In [22]:
network = Network()


In [23]:
from torchsummary import summary

In [24]:
summary(network, [(1,28,28),(1,1,10)])


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 24, 24]             156
            Conv2d-2             [-1, 12, 8, 8]           1,812
            Linear-3                   [-1, 20]             220
            Linear-4                   [-1, 40]             840
            Linear-5                  [-1, 120]          27,960
            Linear-6                   [-1, 60]           7,260
            Linear-7                   [-1, 29]           1,769
Total params: 40,017
Trainable params: 40,017
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.03
Forward/backward pass size (MB): 0.03
Params size (MB): 0.15
Estimated Total Size (MB): 0.22
----------------------------------------------------------------


In [30]:
batch_size=10
train_loader = torch.utils.data.DataLoader(train_set, batch_size)
optimizer = optim.Adam(network.parameters(), lr=0.01)
sigmoid = nn.Sigmoid() # initialize sigmoid layer
lossCal = nn.BCEWithLogitsLoss() # initialize loss function

for epoch in range(4):

    total_loss = 0
    total_correct = 0

    for batch in train_loader: # Get Batch
        images, labels = batch 
        #print("One hot ")

        randomNumTensor = torch.as_tensor(getRandomNumber(batch_size))   
        actualabels=torch.cat((F.one_hot(labels, num_classes=10), F.one_hot(randomNumTensor + labels, num_classes=19)), 1).type(torch.float)  
        print(randomNumTensor)
        print(actualabels)
        print(labels)
        print(torch.as_tensor(F.one_hot(randomNumTensor, num_classes=10)))



        
        predsLabels = network(images,torch.as_tensor(F.one_hot(randomNumTensor, num_classes=10) , dtype=torch.float)) # Pass Batch
                
        #loss = F.cross_entropy(preds, labels) # Calculate Loss
        loss = lossCal(predsLabels, actualabels) # Calculate Loss

        
      
        print(predsLabels)
        print(predsLabels[:,0:10])
        print(predsLabels[:,10:])
        print("ARGMAX Index")
        print(preds[:,0:10].argmax(dim=1))
        print(preds[:,10:].argmax(dim=1))

        print("Actual Label")
        print(labels)
        print(randomNumTensor+labels)
        print("H2")
        optimizer.zero_grad()
        loss.backward() # Calculate Gradients
        optimizer.step() # Update Weights

        total_loss += loss.item()
        total_correct += get_num_correct(preds, labels)
        break

    print(
        "epoch", epoch, 
        "total_correct:", total_correct, 
        "loss:", total_loss
    )

tensor([9, 0, 3, 7, 8, 5, 4, 1, 2, 6])
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0.,